In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
from scipy.io import wavfile
import librosa
import os
import time
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import joblib
import json
from datetime import datetime

# Set style for better plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

class FixedCompressionFramework:
    """Fixed framework with proper stratified sampling"""
    
    def __init__(self, wavelet='db4', dwt_level=3, sample_rate=16000, duration=2.0, random_seed=42):
        self.wavelet = wavelet
        self.dwt_level = dwt_level
        self.sample_rate = sample_rate
        self.duration = duration
        self.random_seed = random_seed
        self.scaler = StandardScaler()
        self.failed_files = []
        
        # Create output directories
        self.create_directories()
    
    def create_directories(self):
        """Create necessary directories"""
        directories = ['results', 'visualizations']
        for dir_name in directories:
            os.makedirs(dir_name, exist_ok=True)
    
    def load_dataset(self, n_files=100):
        """Load dataset with proper sampling"""
        print("Loading dataset...")
        
        base_path = "UrbanSound8K/"
        metadata_path = os.path.join(base_path, "metadata/UrbanSound8K.csv")
        
        if os.path.exists(metadata_path):
            metadata = pd.read_csv(metadata_path)
            
            # Ensure balanced sampling across classes
            file_paths = []
            labels = []
            
            # Get unique classes
            unique_classes = metadata['class'].unique()
            
            # Sample n_files per class (or as many as available)
            files_per_class = max(1, n_files // len(unique_classes))
            
            for class_name in unique_classes:
                class_files = metadata[metadata['class'] == class_name]
                
                # Take min(files_per_class, available_files)
                sample_size = min(files_per_class, len(class_files))
                if sample_size > 0:
                    sampled_files = class_files.sample(sample_size, random_state=self.random_seed)
                    
                    for _, row in sampled_files.iterrows():
                        fold = row['fold']
                        filename = row['slice_file_name']
                        file_path = os.path.join(base_path, f"audio/fold{fold}", filename)
                        file_path = file_path.replace('\\', '/')
                        
                        if os.path.exists(file_path):
                            file_paths.append(file_path)
                            labels.append(class_name)
                        else:
                            # Try alternative path
                            alt_path = os.path.join(base_path, f"fold{fold}", filename)
                            alt_path = alt_path.replace('\\', '/')
                            if os.path.exists(alt_path):
                                file_paths.append(alt_path)
                                labels.append(class_name)
            
            print(f"Loaded {len(file_paths)} files, {len(set(labels))} classes")
            return file_paths, labels
        
        else:
            print("UrbanSound8K not found, creating synthetic dataset...")
            return self.create_synthetic_dataset(n_files)
    
    def create_synthetic_dataset(self, n_files=100):
        """Create synthetic audio files"""
        print(f"Creating {n_files} synthetic audio files...")
        
        temp_dir = "synthetic_audio"
        os.makedirs(temp_dir, exist_ok=True)
        
        file_paths = []
        labels = []
        
        class_names = ['sine', 'noise', 'chirp', 'silence', 'mixed']
        sr = self.sample_rate
        duration = self.duration
        
        files_per_class = max(1, n_files // len(class_names))
        
        for class_idx, label in enumerate(class_names):
            for i in range(files_per_class):
                if label == 'sine':
                    t = np.linspace(0, duration, int(sr * duration))
                    freq = 440 + (class_idx * 100) + (i * 10)
                    audio = 0.5 * np.sin(2 * np.pi * freq * t)
                elif label == 'noise':
                    audio = np.random.randn(int(sr * duration)) * 0.1
                elif label == 'chirp':
                    t = np.linspace(0, duration, int(sr * duration))
                    freq = 100 + 500 * t + (class_idx * 100)
                    audio = 0.3 * np.sin(2 * np.pi * freq * t)
                elif label == 'silence':
                    audio = np.zeros(int(sr * duration))
                else:  # mixed
                    t = np.linspace(0, duration, int(sr * duration))
                    audio = 0.2 * np.sin(2 * np.pi * 440 * t) + \
                            0.1 * np.random.randn(int(sr * duration))
                
                filename = f"{temp_dir}/audio_{label}_{class_idx}_{i}.wav"
                wavfile.write(filename, sr, (audio * 32767).astype(np.int16))
                
                file_paths.append(filename)
                labels.append(label)
        
        print(f"Created {len(file_paths)} synthetic files")
        return file_paths, labels
    
    def extract_features(self, audio, sr):
        """Extract robust features"""
        features = []
        
        # 1. Basic statistics
        features.append(np.mean(audio))
        features.append(np.std(audio))
        features.append(np.max(np.abs(audio)))
        features.append(np.min(audio))
        
        # 2. Zero Crossing Rate
        zcr = librosa.feature.zero_crossing_rate(audio, frame_length=2048, hop_length=512)
        features.append(np.mean(zcr))
        features.append(np.std(zcr))
        
        # 3. Energy
        energy = np.sum(audio ** 2)
        features.append(energy)
        
        # 4. Simple spectral features
        if len(audio) > 10 and np.any(audio != 0):
            try:
                # Simple FFT
                fft = np.fft.fft(audio)
                magnitude = np.abs(fft)
                
                # Take first half (positive frequencies)
                half_len = len(magnitude) // 2
                if half_len > 0:
                    pos_mag = magnitude[:half_len]
                    
                    features.append(np.mean(pos_mag))
                    features.append(np.std(pos_mag))
                    features.append(np.max(pos_mag))
                    
                    # Simple spectral centroid
                    if np.sum(pos_mag) > 0:
                        freqs = np.fft.fftfreq(len(audio), 1/sr)[:half_len]
                        centroid = np.sum(freqs * pos_mag) / np.sum(pos_mag)
                        features.append(centroid)
                    else:
                        features.append(0.0)
                else:
                    features.extend([0.0, 0.0, 0.0, 0.0])
                    
            except:
                features.extend([0.0, 0.0, 0.0, 0.0])
        else:
            features.extend([0.0, 0.0, 0.0, 0.0])
        
        # Ensure fixed size
        target_size = 13
        if len(features) < target_size:
            features.extend([0.0] * (target_size - len(features)))
        
        return np.array(features[:target_size])
    
    def create_measurement_matrix(self, M, N, matrix_type):
        """Create measurement matrix"""
        np.random.seed(self.random_seed)
        
        if matrix_type == 'bernoulli':
            return np.random.choice([-1, 1], size=(M, N)) / np.sqrt(M)
        elif matrix_type == 'gaussian':
            return np.random.randn(M, N) / np.sqrt(M)
        else:  # logistic_henon
            r = 3.99
            a, b = 1.4, 0.3
            x_log = np.random.rand()
            x_hen, y_hen = np.random.rand(), np.random.rand()
            
            matrix = np.zeros((M, N))
            for i in range(M):
                for j in range(N):
                    # Logistic map
                    x_log = r * x_log * (1 - x_log)
                    
                    # Henon map
                    x_hen_new = 1 - a * x_hen**2 + y_hen
                    y_hen = b * x_hen
                    x_hen = x_hen_new
                    
                    # Combine
                    chaotic = (x_log + x_hen) / 2
                    matrix[i, j] = 2 * chaotic - 1
                
                # Reset occasionally
                if i % 100 == 0:
                    x_log = (x_log + np.random.rand()) / 2
                    x_hen = (x_hen + np.random.rand()) / 2
            
            return matrix / np.sqrt(M)
    
    def compress_audio(self, audio, compression_ratio=0.5, matrix_type='bernoulli'):
        """Compress audio using compressed sensing"""
        # Apply DWT
        try:
            coeffs = pywt.wavedec(audio, self.wavelet, level=self.dwt_level)
            coeffs_flat = np.concatenate(coeffs)
        except:
            # If DWT fails, use raw audio (truncated)
            coeffs_flat = audio[:min(len(audio), 1000)]
        
        N = len(coeffs_flat)
        M = max(10, int(N * compression_ratio))  # Minimum 10 measurements
        
        # Generate measurement matrix
        Phi = self.create_measurement_matrix(M, N, matrix_type)
        
        # Compress
        compressed = Phi @ coeffs_flat
        
        # Add statistics
        features = np.concatenate([
            compressed,
            [
                np.mean(compressed),
                np.std(compressed),
                np.max(np.abs(compressed)),
                M / N  # Actual compression ratio
            ]
        ])
        
        return features
    
    def prepare_all_features(self, file_paths, labels, compression_ratio=None, matrix_type=None):
        """Prepare features with proper error handling"""
        print("Extracting features...")
        
        features_list = []
        labels_list = []
        
        for file_path, label in tqdm(zip(file_paths, labels), total=len(file_paths), desc="Processing"):
            try:
                # Load audio
                audio, sr = librosa.load(file_path, sr=self.sample_rate, 
                                        duration=self.duration, mono=True)
                
                # Extract features
                if matrix_type is None:  # Original features
                    features = self.extract_features(audio, sr)
                else:  # Compressed features
                    features = self.compress_audio(audio, compression_ratio, matrix_type)
                
                features_list.append(features)
                labels_list.append(label)
                
            except Exception as e:
                self.failed_files.append((file_path, str(e)))
                # Use zero features as fallback
                if matrix_type is None:
                    features_list.append(np.zeros(13))
                else:
                    features_list.append(np.zeros(50 + 4))  # 50 compressed + 4 stats
                labels_list.append(label)
        
        X = np.array(features_list)
        y = np.array(labels_list)
        
        print(f"Extracted features: {X.shape}")
        return X, y
    
    def run_stratified_experiment(self):
        """Run experiment with proper stratified sampling"""
        print("="*80)
        print("RUNNING STRATIFIED COMPRESSION EXPERIMENT")
        print("="*80)
        
        # Load balanced dataset
        file_paths, labels = self.load_dataset(200)
        
        if len(file_paths) < 20:
            print("Warning: Small dataset, results may vary")
        
        # Get unique classes
        unique_classes = np.unique(labels)
        print(f"Classes: {unique_classes}")
        print(f"Total files: {len(file_paths)}")
        
        # Experiment configurations
        experiments = [
            {'name': 'original', 'matrix_type': None, 'compression_ratio': None},
            {'name': 'bernoulli_50', 'matrix_type': 'bernoulli', 'compression_ratio': 0.5},
            {'name': 'bernoulli_60', 'matrix_type': 'bernoulli', 'compression_ratio': 0.6},
            {'name': 'bernoulli_70', 'matrix_type': 'bernoulli', 'compression_ratio': 0.7},
            {'name': 'gaussian_50', 'matrix_type': 'gaussian', 'compression_ratio': 0.5},
            {'name': 'gaussian_60', 'matrix_type': 'gaussian', 'compression_ratio': 0.6},
            {'name': 'gaussian_70', 'matrix_type': 'gaussian', 'compression_ratio': 0.7},
            {'name': 'logistic_henon_50', 'matrix_type': 'logistic_henon', 'compression_ratio': 0.5},
            {'name': 'logistic_henon_60', 'matrix_type': 'logistic_henon', 'compression_ratio': 0.6},
            {'name': 'logistic_henon_70', 'matrix_type': 'logistic_henon', 'compression_ratio': 0.7},
        ]
        
        results = []
        
        for exp in experiments:
            print(f"\n🔬 Experiment: {exp['name']}")
            
            try:
                # Extract features
                X, y = self.prepare_all_features(
                    file_paths, labels,
                    compression_ratio=exp['compression_ratio'],
                    matrix_type=exp['matrix_type']
                )
                
                # Encode labels once for this experiment
                le = LabelEncoder()
                y_encoded = le.fit_transform(y)
                
                # Stratified split
                if len(np.unique(y_encoded)) > 1:
                    # Use multiple splits for robustness
                    accuracies = []
                    feature_dims = []
                    
                    for split_seed in range(3):  # 3 different splits
                        X_train, X_test, y_train, y_test = train_test_split(
                            X, y_encoded, 
                            test_size=0.2,
                            stratify=y_encoded,
                            random_state=split_seed
                        )
                        
                        # Scale features
                        scaler = StandardScaler()
                        X_train_scaled = scaler.fit_transform(X_train)
                        X_test_scaled = scaler.transform(X_test)
                        
                        # Train Random Forest
                        clf = RandomForestClassifier(
                            n_estimators=100,
                            random_state=self.random_seed,
                            n_jobs=-1
                        )
                        clf.fit(X_train_scaled, y_train)
                        
                        # Predict and evaluate
                        y_pred = clf.predict(X_test_scaled)
                        accuracy = accuracy_score(y_test, y_pred)
                        
                        accuracies.append(accuracy)
                        feature_dims.append(X.shape[1])
                    
                    # Average results
                    avg_accuracy = np.mean(accuracies)
                    avg_features = np.mean(feature_dims)
                    
                    result = {
                        'experiment': exp['name'],
                        'matrix_type': exp['matrix_type'],
                        'compression_ratio': exp['compression_ratio'],
                        'accuracy': avg_accuracy,
                        'accuracy_std': np.std(accuracies),
                        'features': avg_features,
                        'samples': len(X)
                    }
                    
                    results.append(result)
                    
                    print(f"  ✓ Accuracy: {avg_accuracy:.4f} (±{np.std(accuracies):.4f})")
                    print(f"  ✓ Features: {avg_features:.0f}")
                    
                else:
                    print(f"  ✗ Only one class in dataset")
                    
            except Exception as e:
                print(f"  ✗ Error: {str(e)[:100]}...")
                continue
        
        # Save results
        if results:
            results_df = pd.DataFrame(results)
            results_df = results_df.sort_values('accuracy', ascending=False)
            
            results_file = 'results/compression_results_final.csv'
            results_df.to_csv(results_file, index=False)
            
            print(f"\n✅ Results saved to: {results_file}")
            
            # Create visualizations
            self.create_visualizations(results_df)
            
            # Print summary
            self.print_summary(results_df)
            
            return results_df
        else:
            print("\n❌ No valid results obtained")
            return None
    
    def create_visualizations(self, results_df):
        """Create comprehensive visualizations"""
        print("\n📊 Creating visualizations...")
        
        # Filter out original for compressed comparisons
        compressed_df = results_df[results_df['matrix_type'].notnull()].copy()
        original_acc = results_df[results_df['experiment'] == 'original']['accuracy'].values
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        
        # Plot 1: All experiments comparison
        ax = axes[0, 0]
        x_pos = np.arange(len(results_df))
        bars = ax.bar(x_pos, results_df['accuracy'], 
                     yerr=results_df.get('accuracy_std', 0),
                     alpha=0.7, capsize=5)
        
        # Color by matrix type
        colors = {'bernoulli': 'blue', 'gaussian': 'green', 
                 'logistic_henon': 'red', None: 'black'}
        for bar, exp in zip(bars, results_df['experiment']):
            for matrix, color in colors.items():
                if matrix and exp.startswith(matrix):
                    bar.set_color(color)
                    break
            else:
                bar.set_color('black')
        
        ax.set_xticks(x_pos)
        ax.set_xticklabels(results_df['experiment'], rotation=45, ha='right')
        ax.set_ylabel('Accuracy')
        ax.set_title('All Experiments Comparison')
        ax.grid(True, alpha=0.3)
        
        # Plot 2: Matrix type comparison
        ax = axes[0, 1]
        if len(compressed_df) > 0:
            matrix_avg = compressed_df.groupby('matrix_type')['accuracy'].mean()
            matrix_std = compressed_df.groupby('matrix_type')['accuracy'].std()
            
            x_pos = np.arange(len(matrix_avg))
            bars = ax.bar(x_pos, matrix_avg, yerr=matrix_std,
                         alpha=0.7, capsize=5,
                         color=['blue', 'green', 'red'])
            
            if len(original_acc) > 0:
                ax.axhline(y=original_acc[0], color='black', linestyle='--', 
                          label='Original', linewidth=2)
            
            ax.set_xticks(x_pos)
            ax.set_xticklabels(matrix_avg.index)
            ax.set_ylabel('Average Accuracy')
            ax.set_title('Matrix Type Comparison')
            ax.legend()
            ax.grid(True, alpha=0.3)
        
        # Plot 3: Compression ratio effect
        ax = axes[0, 2]
        if len(compressed_df) > 0:
            for matrix_type in compressed_df['matrix_type'].unique():
                matrix_data = compressed_df[compressed_df['matrix_type'] == matrix_type]
                ax.plot(matrix_data['compression_ratio'], matrix_data['accuracy'],
                       marker='o', linewidth=2, label=matrix_type,
                       color=colors[matrix_type])
            
            if len(original_acc) > 0:
                ax.axhline(y=original_acc[0], color='black', linestyle='--',
                          label='Original', linewidth=2)
            
            ax.set_xlabel('Compression Ratio')
            ax.set_ylabel('Accuracy')
            ax.set_title('Effect of Compression Ratio')
            ax.legend()
            ax.grid(True, alpha=0.3)
        
        # Plot 4: Feature dimension vs accuracy
        ax = axes[1, 0]
        scatter = ax.scatter(results_df['features'], results_df['accuracy'],
                           c=results_df['compression_ratio'].fillna(1.0),
                           s=100, alpha=0.7, cmap='viridis')
        
        # Add labels for compressed experiments
        for _, row in compressed_df.iterrows():
            ax.annotate(f"{row['matrix_type'][:3]}_{int(row['compression_ratio']*100)}%",
                       (row['features'], row['accuracy']),
                       fontsize=8, ha='center')
        
        ax.set_xlabel('Feature Dimension')
        ax.set_ylabel('Accuracy')
        ax.set_title('Feature Dimension vs Accuracy')
        ax.grid(True, alpha=0.3)
        plt.colorbar(scatter, ax=ax, label='Compression Ratio')
        
        # Plot 5: Best configurations
        ax = axes[1, 1]
        if len(compressed_df) > 0:
            # Get top 5 compressed configurations
            top5 = compressed_df.nlargest(5, 'accuracy')
            
            bars = ax.bar(range(len(top5)), top5['accuracy'],
                         yerr=top5.get('accuracy_std', 0),
                         alpha=0.7, capsize=5)
            
            # Color bars
            for i, (_, row) in enumerate(top5.iterrows()):
                bars[i].set_color(colors[row['matrix_type']])
            
            ax.set_xticks(range(len(top5)))
            ax.set_xticklabels([f"{row['matrix_type'][:3]}_{int(row['compression_ratio']*100)}%"
                              for _, row in top5.iterrows()], rotation=45, ha='right')
            ax.set_ylabel('Accuracy')
            ax.set_title('Top 5 Compressed Configurations')
            ax.grid(True, alpha=0.3)
            
            # Add value labels
            for i, bar in enumerate(bars):
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                       f'{bar.get_height():.3f}', ha='center', va='bottom')
        
        # Plot 6: Speed vs Accuracy trade-off (estimated)
        ax = axes[1, 2]
        if len(results_df) > 0:
            # Estimate computational complexity
            # Original features are fastest, compressed take longer
            estimated_time = []
            for _, row in results_df.iterrows():
                if row['experiment'] == 'original':
                    estimated_time.append(1.0)  # baseline
                else:
                    # More features = longer time
                    time_factor = row['features'] / 13  # relative to original
                    # Compressed sensing adds overhead
                    if row['matrix_type'] == 'logistic_henon':
                        time_factor *= 1.5  # chaotic maps are slower
                    estimated_time.append(time_factor)
            
            results_df['estimated_time'] = estimated_time
            
            scatter = ax.scatter(results_df['estimated_time'], results_df['accuracy'],
                               c=results_df['compression_ratio'].fillna(1.0),
                               s=100, alpha=0.7, cmap='viridis')
            
            ax.set_xlabel('Estimated Relative Time')
            ax.set_ylabel('Accuracy')
            ax.set_title('Speed vs Accuracy Trade-off')
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('visualizations/comprehensive_results.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("✓ Visualizations saved to: visualizations/comprehensive_results.png")
    
    def print_summary(self, results_df):
        """Print comprehensive summary"""
        print("\n" + "="*80)
        print("EXPERIMENT SUMMARY")
        print("="*80)
        
        # Original performance
        original_row = results_df[results_df['experiment'] == 'original']
        if len(original_row) > 0:
            print(f"\n📈 ORIGINAL FEATURES:")
            print(f"  Accuracy: {original_row['accuracy'].values[0]:.4f}")
            print(f"  Features: {original_row['features'].values[0]:.0f}")
        
        # Best compressed
        compressed_df = results_df[results_df['matrix_type'].notnull()]
        if len(compressed_df) > 0:
            best_compressed = compressed_df.loc[compressed_df['accuracy'].idxmax()]
            
            print(f"\n🏆 BEST COMPRESSED CONFIGURATION:")
            print(f"  Matrix: {best_compressed['matrix_type'].capitalize()}")
            print(f"  Compression: {best_compressed['compression_ratio']*100:.0f}%")
            print(f"  Accuracy: {best_compressed['accuracy']:.4f}")
            print(f"  Features: {best_compressed['features']:.0f}")
            
            if len(original_row) > 0:
                orig_acc = original_row['accuracy'].values[0]
                comp_acc = best_compressed['accuracy']
                accuracy_drop = orig_acc - comp_acc
                feature_reduction = 1 - (best_compressed['features'] / original_row['features'].values[0])
                
                print(f"  Accuracy Drop: {accuracy_drop:.4f}")
                print(f"  Feature Reduction: {feature_reduction*100:.1f}%")
        
        # Best by matrix type
        print(f"\n📊 BEST BY MATRIX TYPE:")
        for matrix_type in compressed_df['matrix_type'].unique():
            matrix_data = compressed_df[compressed_df['matrix_type'] == matrix_type]
            best_matrix = matrix_data.loc[matrix_data['accuracy'].idxmax()]
            print(f"  {matrix_type.capitalize()}: {best_matrix['compression_ratio']*100:.0f}% "
                  f"(Acc: {best_matrix['accuracy']:.4f})")
        
        # Recommendations
        print(f"\n💡 RECOMMENDATIONS:")
        if len(compressed_df) > 0:
            # Find configurations with < 10% accuracy drop
            if len(original_row) > 0:
                orig_acc = original_row['accuracy'].values[0]
                good_configs = compressed_df[compressed_df['accuracy'] >= orig_acc * 0.9]
                
                if len(good_configs) > 0:
                    print("  Configurations with <10% accuracy drop:")
                    for _, row in good_configs.nlargest(3, 'accuracy').iterrows():
                        print(f"    • {row['matrix_type']} {row['compression_ratio']*100:.0f}%: "
                              f"Acc={row['accuracy']:.4f}, Features={row['features']:.0f}")
                else:
                    print("  No configurations with <10% accuracy drop found.")
            
            # Fastest configuration
            fastest = compressed_df.loc[compressed_df['features'].idxmin()]
            print(f"  Fastest (fewest features): {fastest['matrix_type']} "
                  f"{fastest['compression_ratio']*100:.0f}%")
        
        print(f"\n📁 Output files:")
        print(f"  - results/compression_results_final.csv")
        print(f"  - visualizations/comprehensive_results.png")

def main():
    """Main execution"""
    print("🎯 COMPRESSED SENSING AUDIO CLASSIFICATION EXPERIMENT")
    print("="*80)
    
    # Initialize framework
    framework = FixedCompressionFramework(
        wavelet='db4',
        dwt_level=3,
        sample_rate=22050,
        duration=3.0,
        random_seed=42
    )
    
    print("\n🚀 Starting experiment...")
    start_time = time.time()
    
    # Run the complete experiment
    results = framework.run_stratified_experiment()
    
    total_time = time.time() - start_time
    
    if results is not None:
        print("\n" + "="*80)
        print(f"✅ EXPERIMENT COMPLETED IN {total_time/60:.1f} MINUTES!")
        print("="*80)
        
        # Show quick results
        print("\n🎯 QUICK RESULTS:")
        print("-"*40)
        
        original_acc = results[results['experiment'] == 'original']['accuracy'].values
        if len(original_acc) > 0:
            print(f"Original: {original_acc[0]:.4f}")
        
        if len(results) > 1:
            best_compressed = results[results['experiment'] != 'original'].nlargest(1, 'accuracy')
            if len(best_compressed) > 0:
                row = best_compressed.iloc[0]
                print(f"Best Compressed: {row['experiment']} = {row['accuracy']:.4f}")
        
    else:
        print("\n❌ Experiment failed to produce results")
    
    print("\n" + "="*80)
    print("🎉 All done! Check the 'results' and 'visualizations' folders.")
    print("="*80)

if __name__ == "__main__":
    main()

🎯 COMPRESSED SENSING AUDIO CLASSIFICATION EXPERIMENT

🚀 Starting experiment...
RUNNING STRATIFIED COMPRESSION EXPERIMENT
Loading dataset...
Loaded 200 files, 10 classes
Classes: ['air_conditioner' 'car_horn' 'children_playing' 'dog_bark' 'drilling'
 'engine_idling' 'gun_shot' 'jackhammer' 'siren' 'street_music']
Total files: 200

🔬 Experiment: original
Extracting features...


Processing: 100%|████████████████████████████████████████████████████████████████████| 200/200 [00:14<00:00, 13.74it/s]


Extracted features: (200, 13)
  ✓ Accuracy: 0.3250 (±0.0736)
  ✓ Features: 13

🔬 Experiment: bernoulli_50
Extracting features...


Processing: 100%|██████████████████████████████████████████████████████████████████| 200/200 [1:57:58<00:00, 35.39s/it]


  ✗ Error: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dim...

🔬 Experiment: bernoulli_60
Extracting features...


Processing:  30%|███████████████████▌                                             | 60/200 [1:09:13<2:06:59, 54.42s/it]